In [1]:
# Konfigurasi Spark
import os
import sys
from pprint import pprint
from matplotlib import pyplot

# 1. Lokasi di mana Spark diinstal
spark_path = "/Users/sani/Kuliah/Sem8/BigData/spark-2.1.0-bin-hadoop2.7"

# 2. Menentukan environment variable
os.environ['SPARK_HOME'] = spark_path


# 3. Download winutils dari https://github.com/steveloughran/winutils/blob/master/hadoop-2.6.0/bin/winutils.exe?raw=true
#   dan letakkan di dalam folder D:\spark\bin\
#   Lokasi winutils.exe
os.environ['HADOOP_HOME'] = spark_path

# 4. Lokasi Python yang dijalankan --> punya Anaconda
#    Apabila Python yang diinstall hanya Anaconda, maka tidak perlu menjalankan baris ini.
os.environ['PYSPARK_PYTHON'] = sys.executable

# 5. Konfigurasi path library PySpark
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

#############################################################

# from __future__ import print_function

# $example on$
from numpy import array
from math import sqrt
# $example off$

from pyspark import SparkContext
from pyspark import SparkConf
# $example on$
from pyspark.mllib.clustering import KMeans, KMeansModel
# $example off$

if __name__ == "__main__":
    sc = SparkContext(appName="KMeansExample")  # SparkContext


# $example on$
# Load and parse the data
data = sc.textFile("/Users/sani/Kuliah/Sem8/BigData/BigData2017/Tugas4/complete.csv")
# parsedData = data.map(lambda line: array([x for x in line.split(',')]))
    

In [2]:
header=data.first()
# wohead=data.filter(lambda x:x != header)
wohead=data.filter(lambda x:x != header).map(lambda x:x.split(','))
wohead.take(2)

[[u'10/10/49 20:30',
  u'san marcos',
  u'tx',
  u'us',
  u'cylinder',
  u'2700',
  u'45 minutes',
  u'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit',
  u'4/27/04',
  u'29.8830556',
  u'-97.9411111'],
 [u'10/10/49 21:00',
  u'lackland afb',
  u'tx',
  u'',
  u'light',
  u'7200',
  u'1-2 hrs',
  u'1949 Lackland AFB&#44 TX.  Lights racing across the sky &amp; making 90 degree turns on a dime.',
  u'12/16/05',
  u'29.38421',
  u'-98.581082']]

In [3]:
latlang=wohead.map(lambda x: (x[9],x[10]))
latlang.take(5)

[(u'29.8830556', u'-97.9411111'),
 (u'29.38421', u'-98.581082'),
 (u'53.2', u'-2.916667'),
 (u'28.9783333', u'-96.6458333'),
 (u'21.4180556', u'-157.8036111')]

In [4]:
clean=latlang.filter(lambda x: "0" not in x)
clean.take(5)

[(u'29.8830556', u'-97.9411111'),
 (u'29.38421', u'-98.581082'),
 (u'53.2', u'-2.916667'),
 (u'28.9783333', u'-96.6458333'),
 (u'21.4180556', u'-157.8036111')]

In [5]:
yeyee=clean.map(lambda line: array([x for x in line]))
yeyee.take(5)

[array([u'29.8830556', u'-97.9411111'], 
       dtype='<U11'), array([u'29.38421', u'-98.581082'], 
       dtype='<U10'), array([u'53.2', u'-2.916667'], 
       dtype='<U9'), array([u'28.9783333', u'-96.6458333'], 
       dtype='<U11'), array([u'21.4180556', u'-157.8036111'], 
       dtype='<U12')]

In [6]:
yuhu=yeyee.map(lambda x: ' '.join(x[0:2]))
yuhu.take(5)

[u'29.8830556 -97.9411111',
 u'29.38421 -98.581082',
 u'53.2 -2.916667',
 u'28.9783333 -96.6458333',
 u'21.4180556 -157.8036111']

In [7]:
lili=yuhu.map(lambda line: array([x for x in line.split(' ')]))
lili.take(5)

[array([u'29.8830556', u'-97.9411111'], 
       dtype='<U11'), array([u'29.38421', u'-98.581082'], 
       dtype='<U10'), array([u'53.2', u'-2.916667'], 
       dtype='<U9'), array([u'28.9783333', u'-96.6458333'], 
       dtype='<U11'), array([u'21.4180556', u'-157.8036111'], 
       dtype='<U12')]

In [8]:
parsedData=lili.map(lambda x: array([float(x[0]),float(x[1])]))
parsedData.take(5)

[array([ 29.8830556, -97.9411111]),
 array([ 29.38421 , -98.581082]),
 array([ 53.2     ,  -2.916667]),
 array([ 28.9783333, -96.6458333]),
 array([  21.4180556, -157.8036111])]

In [9]:
    # Build the model (cluster the data)
    clusters = KMeans.train(parsedData,5, maxIterations=100, initializationMode="random")
    
    # Evaluate clustering by computing Within Set Sum of Squared Errors
    def error(point):
        center = clusters.centers[clusters.predict(point)]
        return sqrt(sum([x**2 for x in (point - center)]))

    WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print("Within Set Sum of Squared Error = " + str(WSSSE))

    # Save and load model
#     clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
#     sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
#     # $example off$

#     sc.stop()

Within Set Sum of Squared Error = 841854.205586


In [10]:
clusterdata=parsedData.map(lambda point: clusters.predict(point))
clusterdata.take(15)

[2, 2, 3, 2, 0, 1, 3, 4, 1, 1, 0, 1, 4, 1, 1]